# 케라스를 사용한 인공 신경망 소개
* 다층 퍼셉트론(MLP)소개
* 저수준 API를 이용해서 사용자 정의 케라스 컴포넌트를 만들 수 있다(12장)
---
## 생물학적 뉴런에서 인공 뉴런까지
### 생물학적 뉴런
### 뉴런을 사용한 논리연산
### 퍼셉트론(TLU,LTU)
* 1957년에 프랑크 로젠블라트 제안
* 입력의 가중치 합을 계산한 뒤에 계단함수를 적용하여 결과 출력.
* 하나의 TLU는 간단한 선형 이진 분류 가능
* 모든 뉴런이 연결되면 완전연결층 또는 밀집층.
* 입력은 입력층에서 편향특성이 더해저 다음 층으로 간다.
* 퍼셉트론은 오차가 감소되도록 연결을 강화한다.
* 퍼셉트론은 매개변수가 loss="perceptron",learning_rate="constant",eta0=1,penalty=None인 SGDClassifier와 같다.
* 1969년 민스키 교수가 약점을 언급하며 침체기 빠짐!
* 퍼셉트론을 여러층 쌓으면 xor문제를 풀수 있다.

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron
import numpy as np
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron

iris=load_iris()
x=iris.data[:,(2,3)]
y=(iris.target==0).astype(np.int)

per_clf=Perceptron()
per_clf.fit(x,y)

y_pred=per_clf.predict([[2,0.5]])
y_pred

### 다층 퍼셉트론과 역전파
* 입력층 하나와 은닉층이라 불리는 하나 이상의 TLU층과 마지막 출력층으로 구성.
* 입력층에 가까운 층을 하위층(lower layer),출력에 가까운 층을( upper layer)라고 한다.
* 은닉층을 여러개 쌓아 올린 인공 신경망을 심층 신경망(DNN)이라고 한다.
* 1986년 루멜하트,힌턴이 역전파를 소개한다. 효과적으로 그레디언트 계산하는 경사하강법!
* 네트워크를 정방향 역방향 한번 통과하는 것으로 오차의 그레디언트를 계산 할 수 있다. 즉 가중치와 편향값을 수정 할 수 있다.
* 한번에 하나의 미니배치씩 진행하여 전체 훈련세트를 처리한다. 각 반복을 에포크라고 한다.
* 각 미니배치는 네트워크 입력층으로 전달되 첫번째 은닉층으로 보내진다.
* 그 다음 해당 층에 있는 모든 뉴런의 출력을 계산하고 다음층으로 전달하고 마지막까지 간다.(정방향 계싼)
* 그 다음 출력 오차를 측정한다. 다음 각 출력 연결이 이 오차에 기여하는 정도를 계산한다(미적분의 연쇄법칙이 쉽다).
* 오차 그레디언트를 거꾸로 전파함으로 연결이 오차에 기여한 정도를 측정한다.
* 시그모이드,하이퍼볼릭탄젠트,relu 활성화함수!
* 선형함수 두개를 붙이면 선형이다. 따라서 비선형성이 추가되야해서 활성화 함수를 쓴다.
---
### 회귀를 위한 다층 퍼셉트론.
* 출력 뉴런이 하나다.
* 다변량 회귀는 출력 차원마다 출력 뉴런이 하나씩 필요하다. 2D위치를 찾기 위해 출력 2개! 너비와 톺이를 알려면 2개 더!
* 출력이 항상 양수이기 위함으로 RELU와 softplus(log(1+exp(z))활성화 함수가 있다.
* 어떤 범위 안의 값을 예측하고 싶다면 로지스틱 함수나 하이퍼볼릭 탄젠트가 가능하다.
* 손실함수는 보통 MSE이나 이상치가 많으면 평균 절댓값 오차를 사용할 수 있다. 이 둘을 조합한 후버 손실도 있다.
  * 후버 손실은 오차가 입계값(보통 1) 보다 작으면 이차 크면 선형함수이다.
---
### 분류를 위한 다층 퍼셉트론
* 이진 분류에서는 하나의 출력뉴런만 필요
*  3개 이상의 클래스중 하나만을 선택해야 한다면 소프트맥스를 사용.
* 확률 분포를 예측해야 하므로 손실함수는 보통 크로스엔트로피 손실 사용.
---
### 케라스로 다층 퍼셉트론 구현하기.
* 텐서플로와 케라스와 파이토치가 인기있는 딥러닝 라이브러리다!


In [ ]:
import tensorflow as tf
from tensorflow import keras
tf.__version__,keras.__version__

In [ ]:
fashion_mnist=keras.datasets.fashion_mnist
(x_train_full,y_train_full),(x_test,y_test)=fashion_mnist.load_data()
x_val,x_train=x_train_full[:5000]/255.0,x_train_full[5000:]/255.0
y_val,y_train=y_train_full[:5000],y_train_full[5000:]
x_test=x_test/255.0

In [ ]:
x_train_full.shape,x_train_full.dtype

In [ ]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [ ]:
class_names[y_train[0]]

In [ ]:
import matplotlib.pyplot as plt
n_rows=4
n_columns=10
plt.figure(figsize=(n_columns*1.2,n_rows*1.2))
for row in range(n_rows):
  for columns in range(n_columns):
    index=n_columns*row+columns
    plt.subplot(n_rows,n_columns,index+1)
    plt.imshow(x_train[index],cmap="binary",interpolation="nearest")
    plt.axis('off')
    plt.title(class_names[y_train[index]],fontsize=12)
plt.show()

### 시퀀셜 API를 사용하여 모델 만들기

In [ ]:
model=keras.models.Sequential()                      #시퀀셜 모델을 만든다. 간단한 케라스 신경망 모델이다. 순서대로 쌓는다.
model.add(keras.layers.Flatten(input_shape=[28,28])) # 1D배열로 만든다
model.add(keras.layers.Dense(300,activation="relu")) # 뉴런 300개를 가진 은닉층 추가한다. Dense층 마다 가중치 행렬을 관리한다.
model.add(keras.layers.Dense(120,activation="relu")) # 뉴런 120개를 가진 은닉층 추가
model.add(keras.layers.Dense(10,activation="softmax")) # soft맥스를 활성화 함수로 출력층 추가.

In [ ]:
model=keras.models.Sequential([
                               keras.layers.Flatten(input_shape=[28,28]),
                               keras.layers.Dense(300,activation="relu"),
                               keras.layers.Dense(300,activation="relu"),
                               keras.layers.Dense(100,activation="relu"),
                               keras.layers.Dense(10,activation="softmax")
])

In [ ]:
model.summary()

In [ ]:
print(model.layers)

* Dense층은 연결 가중치를 무작위로 초기화하고 편향은 0으로 초기화한다.
* 다른 초기화 방법을 사용하기 위해 층을 만들 때 kernel_initializer와 bias_initializer 매개변수를 설정할 수 있다.

In [ ]:
import numpy as np
import pandas as pd
weights,biases=model.layers[1].get_weights()
weights.shape,biases.shape

##### 모델 컴파일
* loss 는 손실함수 (이진 분류나 다중 레이블 이진 분류를 수행한다면 출력층에 softmax대신 sigmoid함수를 사용하고 binary_crossentropy를 사용한다.)
* 희소한 레이블(클래스의 인덱스)을 원핫 레이블로 변환하려면 keras.utils.categorical()함수를 사용한다!. 반대로 변환하려면 axis=1로 설정하여 np.argmax()를 사용한다.
* optimizer를 sgd로 지정하면 확률적 경사 하강법을 사용하는것이다.

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",optimizer="sgd",metrics=["accuracy"])

##### 모델 훈련과 평가
* fit 메서드 사용.
* 입력 특성,타깃클래스, 에포크 횟수(default=1)를 전달한다. 검증세트(선택사항)
* validation에 0.1로 쓰면 검증에 마지막 10%를 사용한다.
* 훈련세트의 클래스 개수가 편중되어 있다면 fit에 class_weight변수를 지정하는게 좋다. 샘플별로 가중치를 부여한다면 sample_weight 매개변수를 지정한다( 둘다 지정되면 곱한다.)
* fit 메서드의 출력인 History객체는 훈련파라미터(history.params), 수행된 에포크 리스트(history.epoch)가 포함된다.
* 이 객체의 중요한 속성은 에포크가 끝날때마다 훈련 세트와 검증 세트의 손실과 측정한 지표를 담은 history.history이다. 이것을 통해 판다스 데이터 프레임을 만들고 학습 곡선을 만들 수 있다.

In [ ]:
history=model.fit(x_train,y_train,epochs=5,batch_size=5,validation_data=(x_val,y_val))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

In [ ]:
model.evaluate(x_test,y_test)

### 모델을 사용해 예측을 만들기
* predict를 사용해서 예측을하면 각 레이블별 확률이 나온다!

In [ ]:
x_new=x_test[:3]
y_proba=model.predict(x_new)
y_proba.round(2)

In [ ]:
y_pred=model.predict_classes(x_new)
np.array(class_names)[y_pred]

In [ ]:
for i,image in enumerate(x_test[:3]):
  plt.subplot(1,3,i+1)
  plt.imshow(image,cmap="binary")
  plt.title(class_names[y_pred[i]])
  plt.axis("off")
plt.show()

### 시퀀셜 API를 사용하여 회귀용 다층 퍼셉트론 만들기
* 캘리포니아 주택 가격 데이터셋으로 회귀 신경망으로 해결!
* 출력층이 활성화 함수가 없는 하나의 뉴런(하나의 값을 예측하기에) 과 평균 제곱 오차를 사용하는 것.
* 이 데이터셋은 잡음이 많기에 과대적합을 막는 용도로 뉴런수가 적은 은닉층 하나만 쓴다.

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing=fetch_california_housing()
x_train_full,x_test,y_train_full,y_test=train_test_split(housing.data,housing.target)
x_train,x_val,y_train,y_val=train_test_split(x_train_full,y_train_full)

scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_val=scaler.transform(x_val)
x_test=scaler.transform(x_test)

In [ ]:
model=keras.models.Sequential([
                               keras.layers.Dense(30,activation="relu",input_shape=x_train.shape[1:]),
                               keras.layers.Dense(1)
])

model.compile(loss="mean_squared_error",optimizer="adam")
history=model.fit(x_train,y_train,epochs=10,batch_size=10,validation_data=(x_val,y_val))
mse_test=model.evaluate(x_test,y_test)
x_new=x_test[:3]
y_pred=model.predict(x_new)

In [ ]:
pd.DataFrame(history.history).plot()
print(y_pred)

### 함수형 API를 사용해 복잡한 모델 만들기
* 순차적이지 않은 신경망의 한 예는 와이드&딥 신경망이다.
* 2016년 헝쯔 청의 논문에서 소개되엇다.
* 입력의 일부 또는 전체가 출력층에 바로 연결된다.
* 대조적으로 보통 MLP는 모든 층을 데이터가 통과한다.

In [ ]:
input_=keras.layers.Input(shape=x_train.shape[1:])
hidden1=keras.layers.Dense(30,activation="relu")(input_)
hidden2=keras.layers.Dense(30,activation="relu")(hidden1)
concat=keras.layers.Concatenate()([input_,hidden2])
output=keras.layers.Dense(1)(concat)
model=keras.Model(inputs=[input_],outputs=[output])

In [ ]:
model.summary()

In [ ]:
input_A=keras.layers.Input(shape=[5],name="wide_input")
input_B=keras.layers.Input(shape=[6],name="deep_input")
hidden1=keras.layers.Dense(30,activation="relu")(input_B)
hidden2=keras.layers.Dense(30,activation="relu")(hidden1)
concat=keras.layers.concatenate([input_A,hidden2])
output=keras.layers.Dense(1,name="output")(concat)
model=keras.Model(inputs=[input_A,input_B],outputs=[output])

In [ ]:
model.compile(loss="mse",optimizer=keras.optimizers.SGD(lr=1e-3))

x_train_a,x_train_b=x_train[:,:5],x_train[:,2:]
x_val_a,x_val_b=x_val[:,:5],x_val[:,2:]
x_test_a,x_test_b=x_test[:,:5],x_test[:,2:]
x_new_a,x_new_b=x_test_a[:3],x_test_b[:3]

history=model.fit((x_train_a,x_train_b),y_train,epochs=10,batch_size=10,validation_data=((x_val_a,x_val_b),y_val))
mse_test=model.evaluate((x_test_a,x_test_b),y_test)
y_pred=model.predict((x_new_a,x_new_b))

* 여러개의 출력이 필요한 경우
  * 그림의 주요 물체를 분류하고 위치를 알아야 할 경우(회귀와 분류 둘다 사용)
  * 동일한 데이터에서 독립적인 여러 작업을 수행할 때, 얼굴 사진으로 다중 작업 분류(하나의 출력은 얼굴 표정, 다른 출력은 안경을 썼는지)
  * 규제 기법으로 사용하는 경우. 신경망 구조 안에 보조 출력을 추가할 수 있다.

In [ ]:
input_A=keras.layers.Input(shape=[5],name="wide_input")
input_B=keras.layers.Input(shape=[6],name="deep_input")
hidden1=keras.layers.Dense(30,activation="relu")(input_B)
hidden2=keras.layers.Dense(30,activation="relu")(hidden1)
concat=keras.layers.concatenate([input_A,hidden2])
output=keras.layers.Dense(1,name="output")(concat)
aux_output=keras.layers.Dense(1,name="aux_output")(hidden2)
model=keras.Model(inputs=[input_A,input_B],outputs=[output,aux_output])

In [ ]:
model.compile(loss=["mse","mse"],loss_weights=[0.9,0.1],optimizer="sgd") # 주 출력에 더 많은 가중치를 부여한다!

In [ ]:
history=model.fit([x_train_a,x_train_b],[y_train,y_train],epochs=10,batch_size=5,validation_data=([x_val_a,x_val_b],[y_val,y_val]))

In [ ]:
total_loss,main_loss,aux_loss=model.evaluate([x_test_a,x_test_b],[y_test,y_test])
pd.DataFrame(history.history).plot()

In [ ]:
model.summary()

In [ ]:
model.predict([x_new_a,x_new_b])

In [ ]:
model.evaluate([x_test_a,x_test_b],[y_test,y_test])  # 전체 손실, 주손실,보조손실

### 서브클래싱 API로 동적 모델 만들기
* 시퀀셜 API와 함수형 API는 모두 선언적이다. 층과 연결방식을 먼저 정의해야한다.
  * 모델의 구조를 출력-분석이 쉽고 에러를 일찍 발견하기에도 좋다.
* 정적이다는 것은 반복문과 조건문을 포함하고 사이즈를 변환하는 동적 구조를 포함하기에는 문제가 된다.
* 명령형 API를 위한 서브클래싱 API가 필수적이다.
* 함수형 API와 매우 비슷하지만 Input클래스의 객체를 만들 필요가 없다. 대신 call() 메서드의 input 매개변수를 사용한다.
* 주된 차이점은 call()메서드 안에서 원하는 어떤 계산도 사용할 수 있다는 것이다.
* for, if 텐서플로 저수준 연산을 사용할 수 있다.
* 모델 구조가 call메서드 안에 숨겨져 있기 때문에 케라스가 이를 쉽게 분석할 수 없다. 즉 모델을 저장하거나 복사할 수 없다.
* summary()메서드를 호출하면 층의 목록만 나열되고 층 간의 연결 정보를 얻을 수 없습니다. 
* 케라스가 타입과 크기를 미리 확인할 수 없어 실수가 발생하기 쉽습니다.

In [ ]:
class WideAndDeepModel(keras.models.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)
        
    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output

model = WideAndDeepModel(30, activation="relu")

In [ ]:
model.compile(loss=["mse","mse"],loss_weights=[0.9,0.1],optimizer="sgd")
history=model.fit([x_train_a,x_train_b],[y_train,y_train],epochs=10,validation_data=([x_val_a,x_val_b],[y_val,y_val]))
print(model.evaluate((x_test_a,x_test_b),(y_test,y_test)))
print(model.predict((x_new_a,x_new_b)))

### 모델 저장과 복원
* 서브클래싱에서는 model.saver("~~")를 사용할 수가 없다.
* save_weights() 와 load_weights() 메서드를 사용하여 모델 파라미터를 저장하고 복원할 수 있다.
* 그 외에는 모두 수동으로 저장하고 복원해야 한다.

In [ ]:
housing=fetch_california_housing()
x_train_full,x_test,y_train_full,y_test=train_test_split(housing.data,housing.target)
x_train,x_val,y_train,y_val=train_test_split(x_train_full,y_train_full)

scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_val=scaler.transform(x_val)
x_test=scaler.transform(x_test)

x_train_a,x_train_b=x_train[:,:5],x_train[:,2:]
x_val_a,x_val_b=x_val[:,:5],x_val[:,2:]
x_test_a,x_test_b=x_test[:,:5],x_test[:,2:]
x_new_a,x_new_b=x_test_a[:3],x_test_b[:3]

In [ ]:
input_A=keras.layers.Input(shape=[5],name="wide_input")
input_B=keras.layers.Input(shape=[6],name="deep_input")
hidden1=keras.layers.Dense(30,activation="relu")(input_B)
hidden2=keras.layers.Dense(30,activation="relu")(hidden1)
concat=keras.layers.concatenate([input_A,hidden2])
output=keras.layers.Dense(1,name="output")(concat)
aux_output=keras.layers.Dense(1,name="aux_output")(hidden2)
model=keras.Model(inputs=[input_A,input_B],outputs=[output,aux_output])
model.compile(loss=["mse","mse"],loss_weights=[0.9,0.1],optimizer="sgd")
history=model.fit([x_train_a,x_train_b],[y_train,y_train],epochs=10,batch_size=5,validation_data=([x_val_a,x_val_b],[y_val,y_val]))

In [ ]:
model.save("my_keras_model.h5") # 저장

In [ ]:
model=keras.models.load_model("my_keras_model.h5") # 불러오기

In [ ]:
model.predict((x_new_a,x_new_b)) 

In [ ]:
model.save_weights("my_keras_weigh.ckpt") # 가중치 저장 서브클래스

In [ ]:
model.load_weights("my_keras_weight.ckpt") # 가중치 복원 서브클래스

### 콜벡
* 훈련이 몇시간동안 지속되면 훈련 도중 일정 간격으로 체크포인트를 저장해야한다.
* fit 메서드에서 이걸 콜백으로 할 수 있다.

* ModelCheckpoint는 일정한 간격으로 모델의 체크포인트를 저장한다 기본적으로 매 에포크 끝에서 호출된다.

In [ ]:
input_A=keras.layers.Input(shape=[5],name="wide_input")
input_B=keras.layers.Input(shape=[6],name="deep_input")
hidden1=keras.layers.Dense(30,activation="relu")(input_B)
hidden2=keras.layers.Dense(30,activation="relu")(hidden1)
concat=keras.layers.concatenate([input_A,hidden2])
output=keras.layers.Dense(1,name="output")(concat)
aux_output=keras.layers.Dense(1,name="aux_output")(hidden2)
model=keras.Model(inputs=[input_A,input_B],outputs=[output,aux_output])
model.compile(loss=["mse","mse"],loss_weights=[0.9,0.1],optimizer="sgd")

In [ ]:
#save_best_only=True 설정을 하면 최상의 검증 세트 점수에서만 모델을 저장한다.
# early_stopping_cb=keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)를 통해
# 일정 에포크 동안 검증세트에 대한 점수가 향상되지 않으면 훈련을 멈춘다.
# 모델이 향상되지 않으면 자동으로 중지되기 때문에 에포크의 숫자를 크게 지정해도 된다.
checkpoint_cb=keras.callbacks.ModelCheckpoint("my_keras_model.h5",save_best_only=True)
early_stopping_cb=keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history=model.fit([x_train_a,x_train_b],[y_train,y_train],epochs=10,batch_size=5,
                  validation_data=([x_val_a,x_val_b],[y_val,y_val]),callbacks=[checkpoint_cb,early_stopping_cb])

* 사용자 정의 콜백
  * 아래의 콜백은 훈련하는 동안 검증 손실과 훈련 손실의 비율을 출력한다.
  * on_train_begin(),on_train_end(),on_epoch_begint(),on_epoch_end(),on_batch_begin(),on_batch_end() 등도 있다.
  * 평가에 사용하려면 on_test_begin()등으로 바꿀 수 있다.
  * 예측에 사용하려면 on_prdict_begin()등으로 바꿀 수 있다.

In [ ]:
class PrintValTrainRatioCallback(keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs):
    print("\nval/train:{:.2f}".format(logs["val_loss"]/logs["loss"]))


### 텐서보드를 사용해 시각화하기
* 매우 좋은 인터렉티브 시각화 도구이다.
* 훈련하는 동안 학습 곡선을 그리거나 실행간의 학습 곡선을 비교하고 계산 그래프 시각화와 훈련 통계 분석을 수행할 수 있다.
* 모델이 생성한 이미지를 확인하거나 3D에 투영된 복잡한 다차원 데이터를 시각화 하고 자동으로 클러스터링을 해주는 등 많은 기능이 있다.
* 텐서보드를 사용하려면 프로그램을 수정하여 이벤트 파일이라는 특ㄱ별한 이진 로그 파일에 시각화하려는 데이터를 출력해야한다.
* 각각의 이진 데이터 레코드를 서머리라고 부른다.
* 일반적으로 텐서보드 서버가 루트로그 디렉터리를 가리키고 프로그램은 실행할 때마다 다른 서브디렉터리에 이벤트를 기록한다.
* 텐서보드 로그를 위해 사용할 루트 로그 디렉터리를 정의하고 현재 날짜와 시간을 사용해 실행할 때마다 다른 서브디렉터리 경로를 생성하는 간단한 함수를 만든다.

In [ ]:
import os
root_logdir = os.path.join(os.curdir,"my_logs")

def get_run_logdir():
  import time
  run_id=time.strftime("run_%Y_%m_%d-%H_%M_%S")
  return os.path.join(root_logdir,run_id)
run_logdir=get_run_logdir()

In [ ]:
input_A=keras.layers.Input(shape=[5],name="wide_input")
input_B=keras.layers.Input(shape=[6],name="deep_input")
hidden1=keras.layers.Dense(30,activation="relu")(input_B)
hidden2=keras.layers.Dense(30,activation="relu")(hidden1)
concat=keras.layers.concatenate([input_A,hidden2])
output=keras.layers.Dense(1,name="output")(concat)
aux_output=keras.layers.Dense(1,name="aux_output")(hidden2)
model=keras.Model(inputs=[input_A,input_B],outputs=[output,aux_output])
model.compile(loss=["mse","mse"],loss_weights=[0.9,0.1],optimizer="sgd")
# 모델 구성과 컴파일
tensorboard_cb=keras.callbacks.TensorBoard(run_logdir)
history=model.fit([x_train_a,x_train_b],[y_train,y_train],epochs=5,batch_size=20,
                  validation_data=([x_val_a,x_val_b],[y_val,y_val]),callbacks=[tensorboard_cb])
# 이를 실행하면 로그 디렉터리를 생성한다. 훈련하는 동안 이벤트 파일을 만들고 서머리를 기록한다.
# 실행때마다 하나으 ㅣ디렉터리가 생성된다.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs --port=6006
# 주피터 안에서 바로 텐서보드를 실행할 수 있다.

### 신경망 하이퍼파라미터 튜닝하기
* 많은하이퍼 파라미터를 시도하고 교차검증으로 가장 좋은 점수를 확인하는것!
* GridSearchCv나 랜덤서치로 하이퍼 파라미터 공간 탐색
* 케라스 모델을 사이킷런 추정기처럼 보이도록 바꾸어야 한다.


In [ ]:
def build_model(n_hidden=1,n_neurons=30,learning_rate=3e-3,input_shape=[8]):
  model=keras.models.Sequential()
  model.add(keras.layers.InputLayer(input_shape=input_shape))
  for layer in range(n_hidden):
    model.add(keras.layers.Dense(n_neurons,activation="relu"))
  model.add(keras.layers.Dense(1))
  optimizer=keras.optimizers.SGD(learning_rate=learning_rate)
  model.compile(loss="mse",optimizer=optimizer)
  return model
# 사이킷런과 마찬가지로 가능한 하이퍼파라미터에 적절한 기본값을 설정하는 것이 좋다.

In [ ]:
# buildmodel함수를 사용해 kerasRegressor 클래스의 객체를 만든다.
keras_reg=keras.wrappers.scikit_learn.KerasRegressor(build_model)
#이제 일반적인 사이킷런 회귀 추정기처럼 이 객체를 사용할 수 있다.

In [ ]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs={
    "n_hidden":[0,1,2,3],
    "n_neurons":np.arange(1,100),
    "learning_rate":reciprocal(3e-4,3e-2),
}

rnd_search_cv=RandomizedSearchCV(keras_reg,param_distribs,n_iter=10,cv=3)
rnd_search_cv.fit(x_train,y_train,epochs=20,
                  validation_data=(x_val,y_val),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

In [ ]:
rnd_search_cv.best_params_,rnd_search_cv.best_score_

In [ ]:
model=rnd_search_cv.best_estimator_.model

* 페이지 400쪽 굉장히 많은 하이퍼파라미터 튜닝 라이브러리가 있다.
---
### 은닉층 개수
* 복잡한 문제에서는 심층신경망이 파라미터 효율성이 좋다.
* 계층구조는 좋은 솔루션 뿐만 아니라 새로운 데이터에 일반화 되는 능력도 향상시켜준다
  * 얼굴을 인식하는 모델에서 헤어스타일을 인식하게 하려면 첫번째 네트워크의 하위층을 재사용하여 훈련을 시작할 수 있다.
  * 그 층들의 가중치와 편향값으로 초기화하면 저수준 구조를 학습할 필요가 없게된다. 이를 전이학습이라한다.
  * 네트워크는 비슷한 작업에서 뛰어난 성능을 낸 훈련된 네트워크의 일부를 재사용하는게 일반적이다.
---

### 은닉층의 뉴런 개수
* 데이터셋에 따라 다르지만 다른 은닉층보다 첫번째 은닉층을 크게하는게 도움이 된다.
* 과대적합이 시작되기 전까지 점진적으로 뉴런 수를 늘릴 수 있다.
  * 실전에서는 필요한 것보다 더 많은 층과 뉴런을 가진 모델을 선택하고, 그 다음 과대적합되지 않도록 조기종료나 규제기법을 사용하는것이 간단하고 효과적이다.
  * 뉴런의 수가 너무 적으면 유용한 정보를 표현할 충분한 표현 능력을 가지지 못한다.
---

### 학습률,배치크기 그리고 다른 하이퍼 파라미터
##### 학습률
* 일반적으로 최적의 학습률은 최대학습률(훈련 알고리즘이 발산하는)의 절반정도이다.
* 좋은 학습률을 찾는 한가지 방법은 매우 낮은 학습률에서 매우 큰 학습률까지 수백번 반복하여 모델을 훈련하는 것이다.
* 반복마다 일정한 값을 학습률에 곱한다.
* 최적의 학습률은 손실이 다시 상승하는 점보다 조금 아래에 있을 것이다. 일반적으로 상승점보다 10배 낮은 지점.
##### 옵티마이저
* 좋은 옵티마이저를 선택하는것과 그것의 튜닝이 매우 중요!
##### 배치 크기
* 큰 배치 크기를 사용하면 GPU와 같은 하드웨어 가속기를 효율적으로 활용할 수 있다는 장점!
* GPU 램에 맞는 가장 큰 배치 크기를 사용하는 것이 좋다!.
* 큰 배치 크기는 일반화 성능에 영향을 미치지 않고 훈련 시간을 매우 단축한다.
  * 따라서 학습률 예열(작은 학습률 보다 커지게)같은 기법으로 매우 큰 배치 크기를 시도해 보는 것이 좋다. 성능이 만족하지 못하면 작은 배치 크기를 사용해보자!
##### 활성화 함수
* 일반적으로 Relu가 은닉층에 좋은 기본값이다. 출력은 작업에 따라 다르다.
##### 반복횟수
* 보통 튜닝할 필요가 없고 조기종료를 사용한다
  * 레슬리 스미스의 2018년 논문 확인
  